In [48]:
import cv2
import numpy as np
from glob import glob

ratio = 0.08

images = glob('./A7M4/230727 - 31 문덕 아웃리치/Select/*.jpg')

def kor_imread(path):
    img_array = np.fromfile(path, np.uint8)
    return cv2.imdecode(img_array, cv2.IMREAD_COLOR)

def kor_pngread(path):
    img_array = np.fromfile(path, np.uint8)
    return cv2.imdecode(img_array, cv2.IMREAD_UNCHANGED)

hub_black = 'C:/Users/오해성/Pictures/HUB_BLACK.png'
hub_white = 'C:/Users/오해성/Pictures/HUB_WHITE.png'


def AutoWatermark(image, ratio, white, black):
    img = kor_imread(image)
    h_img, w_img, _ = img.shape

    dst = img[
        int(h_img - (h_img * ratio)) : h_img, int(w_img - (w_img * ratio)) : w_img
    ]

    if np.mean(dst)/255 > 0.5:
        logo = kor_pngread(black)
    else:
        logo = kor_pngread(white)

    h_logo, w_logo, _ = logo.shape

    logo = cv2.resize(
        logo,
        (int(w_img * ratio), int(w_img * ratio * h_logo / w_logo)),
        interpolation=cv2.INTER_CUBIC,
    )
    h_logo, w_logo, _ = logo.shape

    alpha = logo[:, :, 3]
    rgb = logo[:, :, :3]

    alpha = np.expand_dims(alpha, axis=2)
    alpha = np.repeat(alpha, 3, axis=2)

    destination = img[h_img - h_logo : h_img, w_img - w_logo : w_img]
    result = cv2.multiply(destination.astype(float), (1 - (alpha / 255)))
    result += cv2.multiply(rgb.astype(float), (alpha / 255))
    result = result.astype(np.uint8)

    img[h_img - h_logo : h_img, w_img - w_logo : w_img] = result

    cv2.imwrite('./test.jpg', img)

In [34]:
images

['./A7M4/230727 - 31 문덕 아웃리치/Select\\230730 Select-9.jpg',
 './A7M4/230727 - 31 문덕 아웃리치/Select\\230730 Select-10.jpg',
 './A7M4/230727 - 31 문덕 아웃리치/Select\\230727-31 문덕아웃리치 (David)-389-1.jpg',
 './A7M4/230727 - 31 문덕 아웃리치/Select\\230730 Select-1.jpg',
 './A7M4/230727 - 31 문덕 아웃리치/Select\\230730 Select-2.jpg',
 './A7M4/230727 - 31 문덕 아웃리치/Select\\230730 Select-3.jpg',
 './A7M4/230727 - 31 문덕 아웃리치/Select\\230730 Select-5.jpg',
 './A7M4/230727 - 31 문덕 아웃리치/Select\\230730 Select-6.jpg',
 './A7M4/230727 - 31 문덕 아웃리치/Select\\230730 Select-7.jpg',
 './A7M4/230727 - 31 문덕 아웃리치/Select\\230730 Select-8.jpg']

In [49]:
AutoWatermark(images[3], ratio, hub_white, hub_black)